In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 21)

### Tables in the process

In [2]:
format_dict = {
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'SHARES':'{:,}',    
    'Q4':'{:.4f}','Q3':'{:.4f}','Q2':'{:.4f}','Q1':'{:.4f}','DIVIDEND':'{:.4f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
}

In [3]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,KBANK,196,2021-12-14,800,136.88,909,1074,2022-01-21,140.00,231.84,16.23,"111,751.93",38,"2,005.39",1.83%,17.55%,91,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [4]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)

sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1074



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,KBANK,1074,196,2021-12-14,800,136.88,226.67,15.87,Sold,"109,746.54",DTD,None


In [5]:
transaction, name, qty, sell_price, buy_price

('S', 'KBANK', -800, 140.0, 136.88)

### In case of buy, retreive buy id from latest buy record (id)

In [40]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,TSE,1105,200,2022-01-21,40000,2.6,215.28,15.07,Active,104230.35,DTD,None


In [41]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'TSE', 40000, 2.6)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [6]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'KBANK'



,name,date,shares,price,active,dividend,period,grade
0,KBANK,2021-11-29,800,136.50,1,3.0000,4,A4


In [7]:
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -800
WHERE name = 'KBANK'



In [8]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [9]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'KBANK'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,KBANK,2021-11-29,0.0,136.5,0.0,0.0,1,3.0,4,A4


### If shares become zero and no longer wants to keep buy record

In [10]:
sqlDel = """
DELETE FROM buy
WHERE name = '%s' AND volbuy = 0
"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM buy
WHERE name = 'KBANK' AND volbuy = 0



In [11]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'KBANK'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,KBANK,2.5000,0.0000,0.5000,0.0000,3.0000,800,2022-05-07,2022-05-07,,0


In [13]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -800
WHERE name = 'KBANK'



In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'KBANK'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,KBANK,2.5000,0.0000,0.5000,0.0000,3.0000,0,2022-05-07,2022-05-07,,0


### If shares become zero and no longer wants to keep dividend record

In [16]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM dividend
WHERE name = 'KBANK' AND shares = 0



In [17]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [18]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'KBANK'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,637,KBANK,153.00,101.00,I,0.00,150.50,"3,733.70",1.68,136.50,800,-17,19,800,0,0,10pct,SET50


In [21]:
buy_target  = 137   # must calculate manually beforehand
sell_target = 144.5
status = 'X'
reason = "5pct"
sell_qty = 800

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -800,
qty = 800, buy_target = 137, sell_target = 144.5, reason = '5pct', status = 'X'
WHERE name = 'KBANK'



In [22]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [23]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'KBANK'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,637,KBANK,153.00,101.00,X,137.00,144.50,"3,733.70",1.68,136.50,800,-17,19,0,0,0,5pct,SET50


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [45]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'TSE'



,name,date,shares,price,active,dividend,period,grade


### In case there is the same stock name in port

In [27]:
new_cost = 10.90 # new average cost must be calculated manually beforehand

sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 12000, price = 10.9
WHERE name = 'EPG'



In [28]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [29]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'EPG'



,name,date,shares,price,active,dividend,period,grade
0,EPG,2021-06-23,"24,000",10.90,1,0.2300,3,A1


### In case of totally new stock

In [48]:
rcd = ['TSE', '2022-01-21', 40000, 2.60, 0, 0, 1, 0.10, '3', 'A3']
rcd

['TSE', '2022-01-21', 40000, 2.6, 0, 0, 1, 0.1, '3', 'A3']

In [50]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sql)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [52]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [30]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'EPG' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,EPG,0.0000,0.1400,0.0000,0.0900,0.2300,"12,000",2021-12-08,2021-12-08,,0


In [31]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 12000
WHERE name = 'EPG'



In [32]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [33]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'EPG' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,EPG,0.0000,0.1400,0.0000,0.0900,0.2300,"24,000",2021-12-08,2021-12-08,,0


### End of Update dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [34]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)


SELECT * 
FROM stocks 
WHERE name = 'EPG'



In [35]:
stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,631,EPG,13.70,7.95,B,10.20,12.90,228.36,0.38,11.20,"3,000",-9,12,"12,000",0,24H,15pct,SET100


In [36]:
new_cost = 10.90
buy_target = 10.60
sell_target = 12.00
status = 'I'
reason = '10pct'
new_buy_qty = 8000
new_sell_qty = 8000
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

(10.9, 10.6, 12.0, 'I', '10pct', 8000, 8000)

In [37]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 12000, 
cost = 10.9, buy_target = 10.6, sell_target = 12.0, status = 'I', 
reason = '10pct', qty = 8000
WHERE name = 'EPG'



In [38]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [39]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'EPG'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,631,EPG,13.70,7.95,I,10.60,12.00,228.36,0.38,10.90,"8,000",-9,12,"24,000",0,24H,10pct,SET100


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process